# Cross Entropy Loss

In [1]:
import csv
import re
from abc import ABC, abstractmethod
import numpy as np

np.random.seed(99)

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = None
        self.gradient_fn = lambda: None
        self.parents = set()

    def gradient(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.gradient()

    def shape(self):
        return self.data.shape

    def size(self):
        return np.prod(self.data.shape[1:])

    def concat(self, other, axis):
        p = Tensor(np.concatenate([self.data, other.data], axis=axis))

        def gradient_fn():
            self.grad, other.grad = np.split(p.grad, [self.data.shape[axis]], axis=axis)

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

In [3]:
class Sequence:

    def __init__(self, tokens, vocabulary_size, batch_size):
        self.tokens = tokens
        self.vocabulary_size = vocabulary_size
        self.batch_size = batch_size

    def __len__(self):  # 3
        return len(self.tokens) - self.batch_size

    def __getitem__(self, index):  # 4
        return (Tensor([self.tokens[index:index + self.batch_size]]),
                Tensor([self.embedding(self.tokens[index + self.batch_size:
                                                   index + self.batch_size + 1])]))

    def embedding(self, index):
        ebd = np.zeros(self.vocabulary_size)
        ebd[index] = 1
        return ebd

In [4]:
class DataLoader:

    def __init__(self, sequence_batch_size):
        self.sequence_batch_size = sequence_batch_size

        self.reviews = []
        self.sentiments = []
        with open('reviews.csv', 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)
            for _, row in enumerate(reader):
                self.reviews.append(row[0])
                self.sentiments.append(row[1])

        split_reviews = []
        for r in self.reviews:
            split_reviews.append(self.clean_text(r.lower()).split())

        self.vocabulary = set(w for r in split_reviews for w in r)
        self.word2index = {w: idx for idx, w in enumerate(self.vocabulary)}
        self.index2word = {idx: w for idx, w in enumerate(self.vocabulary)}
        self.tokens = [[self.word2index[w] for w in r if w in self.word2index] for r in split_reviews]

        self.train()

    @staticmethod
    def clean_text(text):
        txt = re.sub(r'<[^>]+>', '', text)
        txt = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
        return txt

    def encode(self, text):
        words = self.clean_text(text.lower()).split()
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        return " ".join([self.index2word[index] for index in tokens])

    def train(self):
        self.sequences = self.tokens[:-10]

    def eval(self):
        self.sequences = self.tokens[-10:]

    def __len__(self):  # 3
        return len(self.sequences)

    def __getitem__(self, index):  # 4
        return Sequence(self.sequences[index], len(self.vocabulary), self.sequence_batch_size)

In [5]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, *args):
        return self.forward(*args)

    @abstractmethod
    def forward(self, *args):
        pass

    def parameters(self):
        return []

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

In [6]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.in_size = in_size
        self.out_size = out_size

        self.weight = Tensor(np.random.rand(out_size, in_size) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad = p.grad.T @ x.data
            self.bias.grad = np.sum(p.grad, axis=0)
            x.grad = p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias, x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

In [7]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

In [8]:
class Embedding(Layer):

    def __init__(self, vocabulary_size, embedding_size, axis=1):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.axis = axis

        self.weight = Tensor(np.random.rand(embedding_size, vocabulary_size) / vocabulary_size)

    def forward(self, x: Tensor):
        p = Tensor(np.sum(self.weight.data.T[x.data], axis=self.axis))

        def gradient_fn():
            if self.weight.grad is None:
                self.weight.grad = np.zeros_like(self.weight.data)
            self.weight.grad.T[x.data] += p.grad

        p.gradient_fn = gradient_fn
        p.parents = {self.weight}
        return p

    def parameters(self):
        return [self.weight]

In [9]:
class Tanh(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.tanh(x.data))

        def gradient_fn():
            x.grad = p.grad * (1 - p.data ** 2)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

In [10]:
class CELoss:

    def __call__(self, p: Tensor, y: Tensor):
        exp = np.exp(p.data - np.max(p.data, axis=-1, keepdims=True))
        softmax = exp / np.sum(exp, axis=-1, keepdims=True)

        log = np.log(softmax + 1e-10)
        ce = Tensor(0 - np.sum(y.data * log) / len(p.data))

        def gradient_fn():
            p.grad = (softmax - y.data) / len(p.data)

        ce.gradient_fn = gradient_fn
        ce.parents = {p}
        return ce

In [11]:
class SGD:

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def backward(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

In [12]:
class RNN(Layer):

    def __init__(self, vocabulary_size, embedding_size):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size

        self.embedding = Embedding(vocabulary_size, embedding_size)
        self.update = Linear(embedding_size * 2, embedding_size)
        self.hidden = Linear(embedding_size, embedding_size)
        self.output = Linear(embedding_size, vocabulary_size)
        self.tanh = Tanh()

        self.layers = [self.embedding,
                       self.update,
                       self.hidden,
                       self.output,
                       self.tanh]

    def __call__(self, x: Tensor, h: Tensor):
        return self.forward(x, h)

    def forward(self, x: Tensor, h: Tensor):
        if not h:
            h = Tensor(np.zeros((1, self.embedding_size)))

        embedding_feature = self.embedding(x)
        concat_feature = embedding_feature.concat(h, axis=1)
        cell_hidden = self.tanh(self.update(concat_feature))
        hidden_feature = self.tanh(self.hidden(cell_hidden))

        return (self.output(hidden_feature),
                Tensor(hidden_feature.data))

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

In [13]:
LEARNING_RATE = 0.02
BATCHES = 2
EPOCHS = 100

In [14]:
dataset = DataLoader(BATCHES)

model = RNN(len(dataset.vocabulary), 64)

loss = CELoss()
sgd = SGD(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    for i in range(len(dataset)):
        sequence = dataset[i]

        hidden = None
        for i in range(len(sequence)):
            feature, label = sequence[i]

            prediction, hidden = model(feature, hidden)
            error = loss(prediction, label)

            error.gradient()
            sgd.backward()

print(f'Prediction: {prediction.data}')
print(f'Error: {error.data}')

Prediction: [[-5.13066339 -7.10559578  1.35610944 -4.27289184  0.21230653 -0.33386885
   1.26412053  1.36026116 -0.31456681  3.64645227  1.92364639 -1.88466702
   3.34218751  2.06449565  0.85531739 -1.66184536  0.67092624  7.95268759
   1.4362816   4.76526643 -1.07896438  2.35033443 -0.93635163  2.31365828
   0.10717135 -0.53475055  4.13440353  7.42738162  5.70522817 -5.72099065
  -1.65756046  0.26280901  3.59098591 -2.25204049  1.73185552 -6.32490786
  -3.21338918 -0.62252945 -0.89150117 -5.91800005  5.47233206 -1.7250366
   0.52566628 -0.21149416  4.20820198  0.95346949  0.70784692 -2.96418274
   1.13699946  1.93610281 -0.46723858  4.95103067 -2.76901417 -1.99064743
  -3.17515229  0.11227808  5.92521018  1.0050957  -1.51676859  0.77397267
   0.93019431  5.4647143  -4.77009408 -2.48049396 -2.24117002 -2.75017303
   2.53137977 -1.61476491 -4.70746932  0.08742264  2.52281433 -5.32485312
  -0.49220489 -5.38005721 -1.21292494 -2.45516261  1.19597134  0.22576223
  -0.04855919 -0.14165758 -

In [15]:
dataset.eval()

for i in range(len(dataset)):
    sequence = dataset[i]

    feature, label = sequence[0]
    original = sequence.tokens[:BATCHES]
    generated = original.copy()

    hidden = None
    for j in range(len(sequence)):
        feature, label = sequence[j]

        prediction, hidden = model(feature, hidden)
        original.append(sequence.tokens[j + BATCHES])
        generated.append(prediction.data.argmax())

    print(f'original: {dataset.decode(original)}')
    print(f'generated: {dataset.decode(generated)}')

original: worst movie with awful music the actor did a boring job actress director character screenplay scene but or it at by
generated: worst movie the was her the story actress a wonderful job music director character screenplay actress plot its her screenplay they
original: i recommend this film the actress was wonderful and the music was amazing actor director character scene plot by was
generated: i recommend this excellent was actor was excellent recommend the story story i i actress plot screenplay plot his was
original: poor movie the story and actor were terrible i disappointed the director actress character action screenplay scene by is
generated: poor movie time actor and were the i i enjoyed enjoyed effect screenplay character scene screenplay by plot was
original: excellent film i saw this time perfect performance by the actor and actress director screenplay scene character his her
generated: excellent film i enjoyed the time was time recommend the actor actress the soundt